<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети
####Вариант 5:
Просчитать одну итерацию цикла обучения по Δ-правилу однослойной бинарной
неоднородной нейронной сети, состоящей из 2 нейронов и имеющей функции активации: гиперболический тангенс ($k=2$) и пороговую функцию ($Т=0.5$). В качестве обучающей выборки использовать таблицу истинности для операций эквивалентности и конъюнкции (не использовать первую строчку таблицы). Синаптические веса задать случайным образом.

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [28]:
#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#=================================================================
def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

#=================================================================
#функции для формирования таблицы данных
def x1(v):
  return math.sin(math.log(abs(v[0])) / v[2])

def x2(v):
  return math.cos(v[1]**3)

def x3(v):
  return math.sin(math.tan(v[0] / v[1]) - 3**(v[2]))

# def v1(x, v):
#   return (v[0] + math.tan(x[0])**2)

# def v2(x, v):
#   return (0.2 * v[1])

# def v3(x, v):
#   return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x):
  return (x[0] / x[1]) * math.sqrt(abs(x[0]))

def y2(x):
  return math.sqrt(abs(math.sin(x[2])))

#def y3(x, v):
  #return v[2] * x[0] / (x[1] + 0.1)

inputFunctions = [x1, x2, x3]
outputFunctions = [y1, y2]
#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -10, end = 10):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, elem = 100):
  vector = []
  for i in range(elem):
    vector.append(function(generationRandomVector()))
  return vector

#подставление x,v в выходные функции
def addVariants(function, x, elem = 100):
  vector = []
  for i in range(elem):
    vector.append(function(x[i]))
  return vector

In [29]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.1
#Количество эпох
layer = 10
#количество нейронов на каждом слое
#neurons = [2,3,1]
neurons = 2
#количество входов/выходов соответственно
countX = 3
countY = 2

networkCharact = [layer, neurons, learningRate]
inpOutCounts = [countX, countY]

#=================================================================
data = []
for i in range(len(inputFunctions)):
  data.append(createVariants(inputFunctions[i]))
for i in range(len(outputFunctions)):
  data.append(addVariants(outputFunctions[i], np.array(data[0:3]).T))

data_df = pd.DataFrame(np.array(data).T, columns=['x1', 'x2', 'x3', 'y1', 'y2'])

In [ ]:
#data_df

In [30]:
#charactNet = [кол-во слоёв, кол-во нейронов, коэф. обучения]
#sizeSystem = [кол-во входов, кол-во выходов]

def deltaRule(function, data, charactNet, sizeSystem):
  countLayers = charactNet[0]
  countNeurons = charactNet[1]
  nabla = charactNet[2]

  indexOrder = [i for i in range(0, len(data))] #список всех индексов строк таблицы данных

  #si = [0] * sizeSystem[1]
  yi = [0] * sizeSystem[1]
  epsi = [0] * sizeSystem[1]
  level_i = 0
  k = 0

  #временная инициализация весов (только для однослойной сети!)
  wi = [[randomNumber() for i in range(sizeSystem[0])] for j in range(countNeurons)] #синаптические веса

  while(level_i < countLayers): #цикл по ЭПОХАМ!!!!!!!

    #wi = [[randomNumber() for i in range(sizeSystem[0])] for j in range(countNeurons)] #синаптические веса

    print("------------------------------------------------------------------------------------------------------------------")
    print("\tЭпоха №" + str(level_i + 1))
    random.shuffle(indexOrder)

    while(k < len(data)): #цикл по строкам таблицы с данными

      print('================================================')
      print("Матрица весов для " + str(k + 1) + "-й итерации:")
      print(pd.DataFrame(wi))
      print("\nДля X = " + str(np.array(data.iloc[indexOrder[k], 0:sizeSystem[0]]).T) + '\n')

      eps = 0
      si = [0] * sizeSystem[1]

      for i in range(countNeurons): #по нейронам
        for j in range(sizeSystem[0]): #по входам
          si[i] += data[data.columns[j]][indexOrder[k]] * wi[i][j]

        #сделать проверку на сравнение количества нейронов и выходов для последнего слоя!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        yi[i] = function(si[i])
        epsi[i] = data.iloc[indexOrder[k], sizeSystem[0] + i] - yi[i]
        eps += (epsi[i]**2) / countNeurons

      rez_i = pd.DataFrame([si, yi, data.iloc[k, sizeSystem[0]:], epsi], index=['S', 'Y расч.', 'Y теор.', 'EPS'],
                           columns=['neuron ' + str(i+1) for i in range(sizeSystem[1])])

      print(rez_i)
      print("\nСКО: " + str(eps) + "\n\n")

      #вычисление весов для следующей итерации:
      for i in range(countNeurons):
        for j in range(sizeSystem[0]):
          wi[i][j] = wi[i][j] - nabla * epsi[i] * data[data.columns[j]][indexOrder[k]]

      k += 1

    level_i += 1
    k = 0

  #

In [31]:
#table = pd.DataFrame(it.product([0,1], repeat=countX), columns=['X' + str(i + 1) for i in range(countX)])
#table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
#table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

#deltaRule(hyperbolicTangentFunction, table, networkCharact, inpOutCounts)
#deltaRule(sigmoidFunction, table, layer, neurons, learningRate)
#deltaRule(thresholdFunction, table, layer, neurons, learningRate)

deltaRule(hyperbolicTangentFunction, data_df, networkCharact, inpOutCounts)

Выходные данные были обрезаны до нескольких последних строк (5000).
           0         1          2
0 -27.633219 -2.553441 -60.487586
1  -2.036862  4.629770  44.161249

Для X = [-0.12278394  0.47656813  0.31075956]

          neuron 1   neuron 2
S       -16.621069  16.180025
Y расч.  -1.000000   1.000000
Y теор.   0.110146   0.774937
EPS       0.909721  -0.447024

СКО: 0.5137112916568628


Матрица весов для 8-й итерации:
           0         1          2
0 -27.622049 -2.596795 -60.515856
1  -2.042350  4.651074  44.175141

Для X = [-0.83482958 -0.9922737   0.53454941]

         neuron 1   neuron 2
S       -6.712280  20.703672
Y расч. -0.997571   1.000000
Y теор.  1.027193   0.880058
EPS      1.766286  -0.286240

СКО: 1.6008497962501587


Матрица весов для 9-й итерации:
           0         1          2
0 -27.474594 -2.421532 -60.610273
1  -2.066246  4.622671  44.190441

Для X = [-0.14746895  0.99588821 -0.96318877]

          neuron 1   neuron 2
S        60.019209 -37.655366
Y расч.  

In [ ]:
#1-Количество входов и выходов могут быть различны +
#2-Реализовать расчёт промежуточных слоёв

In [ ]:
#Характеристики многослойной сети:
#1- вектор из кол-ва нейронов в каждом слое (в каждом слое м/б разное кол-во нейронов)
#2- кол-во выходов = кол-ву нейронов в крайнем слое
#3- входов может быть сколько угодно +
#4- в цикле: wij: i-номер нейрона, j-номер входа +
#5- размер s, eps равен количеству выходов +

In [ ]:
#1-Добавить условие для последнего слоя, где сравнивается кол-во нейронов и кол-во выходов
#2-